In [48]:
import pandas as pd
rna_data = pd.read_csv("gene_data/rna_common_complete.csv")
rna_data = rna_data.sort_values(by=['sn','period']).reset_index(drop=True)

In [49]:
X_og_shape = rna_data.drop(['sn','group','caarms_status','period'],axis=1).values
X_reshaped = X_og_shape.reshape(len(set(rna_data['sn'])), 3, X_og_shape.shape[1])
labels_group = rna_data[rna_data['period'] == 24]['group'].values
labels = [0 if i == 'C' else 1 for i in labels_group]

In [50]:
import torch
import numpy as np
from neucube.utils import SNR
from neucube.utils import interpolate
from neucube.encoder import Delta

ratios = SNR(X_reshaped[:,0,:], labels)
top_idx = torch.argsort(ratios, descending=True)[0:20]
X_reshaped_topidx = X_reshaped[:,:,top_idx]
interpolated_X = interpolate(X_reshaped_topidx, num_points=104)

encoder = Delta(threshold=0.008)
X = encoder.encode_dataset(interpolated_X)
y = torch.tensor(labels)

In [51]:
import nevergrad as ng
import numpy as np
from functools import partial
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex
from tqdm import tqdm

def objective_function(res_, X_stimuli, labels, sampler, params):
    #a_exc, b_exc, c_exc, d_exc, a_inh, b_inh, c_inh, d_inh = params
    a_exc, b_exc, c_exc, d_exc = params
    res_.set_exc_parms(a=a_exc, b=b_exc, c=c_exc, d=d_exc)
    #res_.set_inh_parms(a=a_inh, b=b_inh, c=c_inh, d=d_inh)
    out_spikes = res_.simulate(X_stimuli, mem_thr=30, train=False, verbose=False)
    #sampler = TemporalBinning(bin_size=10)
    state_vec = sampler.sample(out_spikes)
    state_vec_SNR = SNR(state_vec,labels)
    sv_top_feat = torch.argsort(state_vec_SNR, descending=True)[0:100]
    state_vec_top = state_vec[:,sv_top_feat]
    return SeparationIndex(state_vec_top, labels).item()

def run_opt(optimizer, objective_):
    for i in range(optimizer.budget):
        x = optimizer.ask()
        loss = -objective_(params=x.value)
        optimizer.tell(x, loss)
        if (i + 1) % 2 == 0:
            print(f"Iteration {i + 1}/{optimizer.budget}, Current loss: {loss}")

def train_dyanmics(reservoir_, X_, y_, sampler_):
    parametrization = ng.p.Tuple(
        ng.p.Scalar(init=reservoir_.a.cpu()[reservoir_.exc_n[0].item()]),
        ng.p.Scalar(init=reservoir_.b.cpu()[reservoir_.exc_n[0].item()]), 
        ng.p.TransitionChoice(list(range(-65,-46))),
        ng.p.TransitionChoice(list(range(2,9))),

        # ng.p.Scalar(init=reservoir_.a.cpu()[reservoir_.inh_n[0].item()]),
        # ng.p.Scalar(init=reservoir_.b.cpu()[reservoir_.inh_n[0].item()]), 
        # ng.p.TransitionChoice(list(range(-65,-46))),
        # ng.p.TransitionChoice(list(range(2,9))),
        )

    parametrization[0].set_bounds(lower=0.01, upper=0.5)
    parametrization[1].set_bounds(lower=0.2, upper=0.75)
    # parametrization[4].set_bounds(lower=0.01, upper=0.5)
    # parametrization[5].set_bounds(lower=0.2, upper=0.75)

    optimizer = ng.optimizers.PortfolioDiscreteOnePlusOne(parametrization=parametrization, budget=1)
    partial_objective_function = partial(objective_function, res_=reservoir_, X_stimuli=X_, labels=y_, sampler=sampler_)

    run_opt(optimizer, partial_objective_function)
    recommendation = optimizer.provide_recommendation()
    #optimal_a_exc, optimal_b_exc, optimal_c_exc, optimal_d_exc, optimal_a_inh, optimal_b_inh, optimal_c_inh, optimal_d_inh = recommendation.value
    return recommendation.value

In [52]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import svm, metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from neucube import IzhReservoir
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex

num_folds = 10
kf = KFold(n_splits=num_folds)
sampler = TemporalBinning(bin_size=10)

true_labels = []
predicted_labels = []
separation_values = []
accuracy_values = []
mcc_values = []

for train_index, test_index in tqdm(kf.split(X)):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    izh_res = IzhReservoir(inputs=X.shape[2], c=0.7, l=0.18, input_conn_prob=0.85)
    izh_res.set_exc_parms(a=0.06, b=0.55, c=-55, d=3) #initial values
    izh_res.set_inh_parms(a=0.1, b=0.2, c=-65, d=2) #initial values

    opt_a_exc, opt_b_exc, opt_c_exc, opt_d_exc = train_dyanmics(izh_res, X_train_fold, y_train_fold, sampler)
    #opt_a_exc, opt_b_exc, opt_c_exc, opt_d_exc, opt_a_inh, opt_b_inh, opt_c_inh, opt_d_inh = train_dyanmics(izh_res, X_train_fold, y_train_fold, sampler)
    izh_res.set_exc_parms(a=opt_a_exc, b=opt_b_exc, c=opt_c_exc, d=opt_d_exc)
    #izh_res.set_inh_parms(a=opt_a_inh, b=opt_b_inh, c=opt_c_inh, d=opt_d_inh)    
    X_train_opt_spike = izh_res.simulate(X_train_fold, mem_thr=30, train=False, verbose=False)
    X_test_opt_spike = izh_res.simulate(X_test_fold, mem_thr=30, train=False, verbose=False)
    X_train_state_vec = sampler.sample(X_train_opt_spike)
    X_test_state_vec = sampler.sample(X_test_opt_spike)

    param_grid = {'C': [2, 3, 4, 5, 6, 7, 8], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear', 'poly']}
    svm_model = svm.SVC()
    mcc_scorer = make_scorer(metrics.matthews_corrcoef)
    grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=10, scoring={'accuracy': 'accuracy', 'mcc': mcc_scorer}, refit='mcc')
    grid_search.fit(X_train_state_vec, y_train_fold)
    y_pred = grid_search.best_estimator_.predict(X_test_state_vec)

    true_labels.extend(y_test_fold)
    predicted_labels.extend(y_pred)
    separation_values.extend([SeparationIndex(X_train_state_vec, y_train_fold), SeparationIndex(X_test_state_vec, y_test_fold)])
    accuracy_values.append(accuracy_score(y_test_fold, y_pred))
    mcc_values.append(metrics.matthews_corrcoef(y_test_fold, y_pred))

0it [00:00, ?it/s]c:\Users\singh\miniconda3\Lib\site-packages\nevergrad\parametrization\_datalayers.py:107: NevergradRuntimeWarning: Bounds are 0.49 sigma away from each other at the closest, you should aim for at least 3 for better quality.
  warnings.warn(
c:\Users\singh\miniconda3\Lib\site-packages\nevergrad\parametrization\_datalayers.py:107: NevergradRuntimeWarning: Bounds are 0.55 sigma away from each other at the closest, you should aim for at least 3 for better quality.
  warnings.warn(
c:\Users\singh\Documents\GitHub\NeuCube-Izh\neucube\utils\utils.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Xin = torch.tensor(Xin, dtype=torch.float32)
c:\Users\singh\Documents\GitHub\NeuCube-Izh\neucube\utils\utils.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTenso

In [53]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
mcc = metrics.matthews_corrcoef(true_labels, predicted_labels)
print("10-Fold Cross-Validation Accuracy:", accuracy)
print("10-Fold Cross-Validation MCC:", mcc)
print(confusion_matrix(true_labels, predicted_labels))
print(accuracy_values)
print(mcc_values)
print(separation_values)

10-Fold Cross-Validation Accuracy: 0.808695652173913
10-Fold Cross-Validation MCC: 0.6143546104361061
[[52 12]
 [10 41]]
[0.9166666666666666, 1.0, 0.9166666666666666, 0.8333333333333334, 0.8333333333333334, 0.9090909090909091, 0.7272727272727273, 0.45454545454545453, 0.7272727272727273, 0.7272727272727273]
[0.0, 0.0, 0.0, -0.09090909090909091, 0.7071067811865476, 0.0, 0.24056261216234406, 0.28867513459481287, 0.0, 0.24056261216234406]
[tensor(0.0118), tensor(0.), tensor(0.0121), tensor(0.), tensor(0.0125), tensor(0.), tensor(0.0127), tensor(0.1199), tensor(0.0135), tensor(0.0880), tensor(0.0129), tensor(0.), tensor(0.0122), tensor(0.2056), tensor(0.0149), tensor(0.0691), tensor(0.0134), tensor(0.), tensor(0.0132), tensor(0.1070)]


In [54]:
#pd.DataFrame({'True_Labels': np.array(true_labels), 'Predicted_Labels': np.array(predicted_labels)}).to_csv('results.csv', index=False)